In [ ]:
import sqlite3
import pandas as pd
import os

In [ ]:
conn=sqlite3.connect("SQL DB.db")

In [ ]:
#csv to sql
def csv_to_sql(path: str)->str:
  table_name=os.path.splitext(os.path.basename(path))[0]
  df=pd.read_csv(path)
  dtype_map={"int64": "INTEGER", "str": "TEXT","object": "TEXT","bool": "INTEGER","float":"Real","None":"NULL"}
  cols=""

  for i,c in enumerate(list(df.columns)):
    cols+=c +" "+ dtype_map[str(df[c].dtype)]
    if (len(df.columns)-1)!=i:
      cols+=",\n"

  cursor=conn.cursor()
  cursor.execute(f"""
  create table if not exists "{table_name}" (
  {cols}
  );""")

  f=cursor.execute(f"""select * from "{table_name}";""")
  data=cursor.fetchall()
  if len(data)==0:
    rows=""
    for i in range(df.shape[0]):
      row = tuple(df.to_numpy(dtype=object)[i])
      rows+=f"{row}"
      if (df.shape[0]-1)!=i:
        rows+=", \n"
    cursor.execute(f"""
    insert into "{table_name}" values {rows};""")

  return "Database Created!",table_name

In [ ]:
csv_to_sql("/content/check.csv")

'Database Created!'

In [ ]:
cursor=conn.cursor()
cursor.execute(f"""select * from "check";""")
x=cursor.fetchall()
print(x)

[(1, 'Ramesh', 32, 2000, 'Maryland', 'USA'), (2, 'Mukesh', 40, 5000, 'New York', 'USA'), (3, 'Sumit', 45, 4500, 'Muscat', 'Oman'), (4, 'Kaushik', 25, 2500, 'Kolkata', 'India'), (5, 'Hardik', 29, 3500, 'Bhopal', 'India'), (6, 'Komal', 38, 3500, 'Saharanpur', 'India'), (7, 'Ayush', 25, 3500, 'Delhi', 'India'), (1, 'Ramesh', 32, 2000, 'Maryland', 'USA'), (2, 'Mukesh', 40, 5000, 'New York', 'USA'), (3, 'Sumit', 45, 4500, 'Muscat', 'Oman'), (4, 'Kaushik', 25, 2500, 'Kolkata', 'India'), (5, 'Hardik', 29, 3500, 'Bhopal', 'India'), (6, 'Komal', 38, 3500, 'Saharanpur', 'India'), (7, 'Ayush', 25, 3500, 'Delhi', 'India')]


# LLM

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip -q install openai

In [ ]:
from langchain_openai import OpenAI
api="xy"#open ai api key
llm=OpenAI(api_key=api)

# SQL Functions

In [ ]:
!pip install langchain_community #database instantiation
!pip install langchain-experimental #contains sqldatabase chain
from langchain_community.utilities import SQLDatabase #to instantiate database
from langchain_experimental.sql import SQLDatabaseChain #to connect model to database to return data
from langchain.chains import create_sql_query_chain #to connect model to database to return data

  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
Using cached langchain_experimental-0.3.4-py3-none-any.whl (209 kB)


In [ ]:
#2. input to query
def input_to_sql(input:str,table_name:str)->str:
  db=SQLDatabase.from_uri("sqlite:///SQl DB.db") #connect to db
  query_chain=create_sql_query_chain(llm,db) #set to return query
  # table_name = os.path.splitext(os.path.basename(path))[0] #table name
  query=query_chain.invoke({"question": f"Using the table '{table_name}', {input}"})
  return query

In [ ]:
#3. query to answer
def query_to_answer(query: str)-> str:
  cursor=conn.cursor()
  cursor.execute(query)
  answer=cursor.fetchall()
  return answer

# Test

In [ ]:
def final(path,input):
  # csv_to_sql(path)
  query=input_to_sql(input,path)
  print(query)
  answer=query_to_answer(query)
  result=f""" Question: {input}

  Answer: {answer}"""
  return result

In [ ]:
input="show name and age for each"
ans=final("/content/check.csv",input)
print(ans)

SELECT "name", "age" FROM "check" LIMIT 5;
 Question: show name and age for each

  Answer: [('Ramesh', 32), ('Mukesh', 40), ('Sumit', 45), ('Kaushik', 25), ('Hardik', 29)]


# Human Readable Format

In [ ]:
def response(ans):
  prompt = f"""
  Act as a Business Strategy Analyst.
  You will be given a question and its answer (context).
  Your task is to explain in a structured manner what this data means from a business perspective and how it can be leveraged.
  {ans}
  """
  res=llm.invoke(prompt)
  result=f"""
  {ans}

  {res}
  """
  return result

In [ ]:
x=response(ans)
print(x)


   Question: show name and age for each

  Answer: [('Ramesh', 32), ('Mukesh', 40), ('Sumit', 45), ('Kaushik', 25), ('Hardik', 29)]

  
From a business perspective, this data shows the names and corresponding ages of five individuals. This information can be leveraged by businesses in various ways.

1. Identifying target demographics:
By knowing the names and ages of these individuals, businesses can gain insights into their target demographics. For example, if the business is a clothing brand, they can use this data to understand the age groups they should cater to and the types of clothing preferences that may align with these age groups.

2. Personalizing marketing strategies:
Having this data can also help businesses personalize their marketing strategies. By knowing the names of their customers, businesses can address them directly in marketing campaigns, creating a more personalized and engaging experience. Similarly, knowing the ages of their customers can help businesses tailo

# Gradio

In [ ]:
#func for gradio
def func(path,input,chat_history):
  ans=final(path,input)
  result=response(ans)
  chat_history.append((input,result))
  return "",chat_history

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:

  path=gr.State() #an object to hold database
  chatbot=gr.Chatbot(label="SQL Query Bot") #chatbot

  with gr.Tab("Create Database"):
    file_input=gr.File(label="Upload CSV file")
    btn=gr.Button("Create Database")
    status=gr.Textbox(label="Status")

    btn.click(csv_to_sql, inputs=file_input, outputs=[status,path])

  with gr.Tab("Query Explainer Chatbot"):
      question_input = gr.Textbox(label="Ask a Question")
      question_input.submit(response, inputs=[path,question_input,chatbot], outputs=[question_input,chatbot])

  clear=gr.ClearButton([question_input,chatbot])

demo.launch(debug=True)

# Agent

In [ ]:
def human_readable(input: str,answer: str)->str:
  ans=f""" Question: {input}

  Answer: {answer}"""

  prompt = f"""
  Act as a Business Strategy Analyst.
  You will be given a question and its answer (context).
  Your task is to explain in a structured manner what this data means from a business perspective and how it can be leveraged.
  {ans}
  """
  res=llm.invoke(prompt)
  return res

In [ ]:
def extract(user_question):
  input=user_question.split("path=")[0]
  path=user_question.split("path=")[1]
  return input,path

In [ ]:
! pip install tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
from langchain.agents import initialize_agent, Tool

In [ ]:
tools= [
    # Tool(name="Extract",func=extract,description="This function extracts input and path from the user_question for other functions."),
    Tool(name="Generate_Table",func=csv_to_sql,description="This function creates a table in database from the given file. The function takes one paramter path, which represents path of the file."),
    Tool(name="Generate_Query",func=input_to_sql, description="This function generates a SQL query based on the user's input question and the table name. The function takes two parameters: 'input' representing the user's question and 'table_name' representing the name of the table. This function must be passed the table_name from Generate_Table function."),
    Tool(name="Fetch_Data",func=query_to_answer,description="This function executes a given SQL query to fetch data from the database.This function takes only one parameter: 'query', which is the generated SQL query based on the user question."),
    Tool(name="Explain_data",func=human_readable,description="This function explains the fetched data in a human readable format from a business perspective. The function takes two parameters: 'input' which is the user_question and 'answer' which is the data fetched using the query_to_answer function.")
]

In [ ]:
agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True
)

In [ ]:
user_question="""Find the name of people that start with M.
 path= /content/check.csv
"""
# path="/content/check.csv"
agent.run(user_question)



> Entering new AgentExecutor chain...
 We need to retrieve data from the CSV file and then filter it based on the given condition.
Action: Generate_Table
Action Input: path

FileNotFoundError: [Errno 2] No such file or directory: 'path'